In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

import pandas as pd
import openpyxl
import xgboost as xgb
from sklearn import metrics
import sklearn
import shap
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
data_location = os.getcwd()+"/../../1.raw/real_7_MS/Metabolic Syndrome.csv"

data = pd.read_csv(data_location)  


In [3]:
data = data.drop(columns=['seqn'])
data = data.dropna()

In [4]:
data = pd.get_dummies(data, columns=['Race'], prefix='Race')
data = pd.get_dummies(data, columns=['Sex', 'Marital'], prefix=['Sex', 'Marital'])

In [5]:
data.columns

Index(['Age', 'Income', 'WaistCirc', 'BMI', 'Albuminuria', 'UrAlbCr',
       'UricAcid', 'BloodGlucose', 'HDL', 'Triglycerides', 'MetabolicSyndrome',
       'Race_Asian', 'Race_Black', 'Race_Hispanic', 'Race_MexAmerican',
       'Race_Other', 'Race_White', 'Sex_Female', 'Sex_Male',
       'Marital_Divorced', 'Marital_Married', 'Marital_Separated',
       'Marital_Single', 'Marital_Widowed'],
      dtype='object')

In [6]:
print(np.sum(data['Marital_Divorced']))
print(np.sum(data['Marital_Married']))
print(np.sum(data['Marital_Separated']))
print(np.sum(data['Marital_Single']))
print(np.sum(data['Marital_Widowed']))

219
1098
88
460
144


In [7]:
print(np.sum(data['Race_Asian']))
print(np.sum(data['Race_Black']))
print(np.sum(data['Race_Hispanic'])) 
print(np.sum(data['Race_MexAmerican']))
print(np.sum(data['Race_Other']))
print(np.sum(data['Race_White']))

295
462
198
198
50
806


In [8]:
data = data.drop(columns=['Marital_Separated'])
data = data.drop(columns=['Race_Other'])


In [9]:
label = data['MetabolicSyndrome'].to_numpy()
features = data.drop(columns=['MetabolicSyndrome']).to_numpy()



In [10]:
data

,Age,Income,WaistCirc,BMI,Albuminuria,UrAlbCr,UricAcid,BloodGlucose,HDL,Triglycerides,...,Race_Black,Race_Hispanic,Race_MexAmerican,Race_White,Sex_Female,Sex_Male,Marital_Divorced,Marital_Married,Marital_Single,Marital_Widowed
0,22,8200.0,81.0,23.3,0,3.88,4.9,92,41,84,...,0,0,0,1,0,1,0,0,1,0
1,44,4500.0,80.1,23.2,0,8.55,4.5,82,28,56,...,0,0,0,1,1,0,0,1,0,0
2,21,800.0,69.6,20.1,0,5.07,5.4,107,43,78,...,0,0,0,0,0,1,0,0,1,0
3,43,2000.0,120.4,33.3,0,5.22,5.0,104,73,141,...,1,0,0,0,1,0,0,0,1,0
5,80,300.0,112.5,28.5,0,9.79,4.8,105,47,100,...,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2394,31,2500.0,74.0,20.6,0,2.00,6.7,95,64,81,...,0,0,0,0,0,1,0,1,0,0
2395,65,5400.0,98.5,29.4,0,5.51,6.7,114,49,165,...,0,0,1,0,1,0,0,1,0,0
2398,28,800.0,100.8,29.4,0,2.78,6.2,99,47,84,...,0,0,1,0,0,1,0,0,1,0
2399,27,8200.0,106.6,31.3,0,4.15,6.2,100,41,124,...,0,0,1,0,0,1,0,1,0,0


In [11]:
print(features)

[[2.200e+01 8.200e+03 8.100e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 [4.400e+01 4.500e+03 8.010e+01 ... 1.000e+00 0.000e+00 0.000e+00]
 [2.100e+01 8.000e+02 6.960e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 ...
 [2.800e+01 8.000e+02 1.008e+02 ... 0.000e+00 1.000e+00 0.000e+00]
 [2.700e+01 8.200e+03 1.066e+02 ... 1.000e+00 0.000e+00 0.000e+00]
 [6.000e+01 6.200e+03 1.066e+02 ... 0.000e+00 1.000e+00 0.000e+00]]


In [12]:
print(label)

[0 0 0 ... 0 1 1]


In [13]:
scaler = MinMaxScaler()
scaler.fit(features)
features = scaler.transform(features)

In [14]:
np.save("features.npy",features)
np.save("label.npy",label)

In [15]:
features.shape

(2009, 21)

In [18]:
print(1-np.sum(label)/label.shape[0])

0.6455948232951717
